In [1]:
import pandas as pd
import numpy as np


In [2]:
#carregamento dos dados
df_credito = pd.read_csv('train.csv')
df_credito.columns = df_credito.columns.str.lower()

C:\Users\Vagner\AppData\Local\Temp\ipykernel_3156\630524447.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_credito = pd.read_csv('train.csv')


Feature Selection

In [3]:
#dropar as colunas de identificador único e que não serão utilizadas (month)
df_credito = df_credito.drop(columns=['id','customer_id', 'month', 'name', 'ssn'])

Tratamento dos dados categóricos

In [4]:
#tratamento da coluna occupation
df_credito['occupation'] = df_credito['occupation'].replace('_______', 'Unknown Occupation')

#separação da coluna occupation em colunas independentes e binárias
occupation_dummies = df_credito['occupation'].str.get_dummies()
occupation_dummies.columns = occupation_dummies.columns.str.lower()

#associação da nova coluna ao dataframe
df_credito = pd.concat([df_credito, occupation_dummies], axis=1)
df_credito = df_credito.drop(columns=['occupation'])

In [5]:
#tratamento da coluna payment_behaviour
df_credito ['payment_behaviour'] = df_credito['payment_behaviour'].replace('!@9#%8', 'Unknown Payment Behavior')

#separação da coluna payment_behaviour em colunas independentes e binárias
payment_behaviour_dummies = df_credito['payment_behaviour'].str.get_dummies()
payment_behaviour_dummies.columns = payment_behaviour_dummies.columns.str.lower()

#associação da nova coluna ao dataframe
df_credito = pd.concat([df_credito, payment_behaviour_dummies], axis=1)
df_credito = df_credito.drop(columns=['payment_behaviour'])

In [6]:
#tratamento da coluna credit_mix
df_credito ['credit_mix'] = df_credito['credit_mix'].replace('_', 'Unknown Credit Mix ')

#separação da coluna credit_mix em colunas independentes e binárias
credit_mix_dummies = df_credito['credit_mix'].str.get_dummies()
credit_mix_dummies.columns = credit_mix_dummies.columns.str.lower()

#associação da nova coluna ao dataframe
df_credito = pd.concat([df_credito, credit_mix_dummies], axis=1)
df_credito = df_credito.drop(columns=['credit_mix'])

In [7]:
#tratamento da coluna payment_of_min_amount
df_credito['payment_of_min_amount'] = df_credito['payment_of_min_amount'].replace('NM', 'NM payment of min amount')
df_credito['payment_of_min_amount'] = df_credito['payment_of_min_amount'].replace('No', 'No payment of min amount')
df_credito['payment_of_min_amount'] = df_credito['payment_of_min_amount'].replace('Yes', 'Yes payment of min amount')

#separação da coluna payment_of_min_amount em colunas independentes e binárias
payment_of_min_amount_dummies = df_credito['payment_of_min_amount'].str.get_dummies()
payment_of_min_amount_dummies.columns = payment_of_min_amount_dummies.columns.str.lower()

#associação da nova coluna ao dataframe
df_credito = pd.concat([df_credito, payment_of_min_amount_dummies], axis=1)
df_credito = df_credito.drop(columns=['payment_of_min_amount'])

In [8]:
#tratamento da coluna type_of_loan
df_credito['type_of_loan'] = df_credito['type_of_loan'].fillna('Unknown Loan')
df_credito['type_of_loan'] = df_credito['type_of_loan'].replace(', ', ',')
df_credito['type_of_loan'] = df_credito['type_of_loan'].replace('and ', '')

#separando occupation em colunas independentes e binárias
loan_dummies = df_credito['type_of_loan'].str.get_dummies(',')
loan_dummies.columns = loan_dummies.columns.str.lower() #vamos usar essa coluna lá em baixo para tratar a coluna num_of_loan

#associar a nova coluna ao dataframe
df_credito = pd.concat([df_credito, loan_dummies], axis=1)
df_credito = df_credito.drop(columns=['type_of_loan'])

Tratamento dos dados numéricos

In [9]:
#tratamento dos valores que possuem '_' no final das colunas numéricas
df_credito = df_credito.applymap(lambda x: str(x).rstrip('_'))
df_credito.replace('nan', np.nan, inplace=True)

In [10]:
#transformar colunas numéricas em numéricas, de fato
def to_numeric(dataframe, columns):
    for column_name in columns:
        dataframe[column_name] = pd.to_numeric(dataframe[column_name], errors='coerce')
    return dataframe


numeric_columns = ['age', 'annual_income', 'monthly_inhand_salary', 'num_bank_accounts', 'num_credit_card',
                    'interest_rate', 'num_of_loan', 'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
                    'num_credit_inquiries', 'outstanding_debt', 'credit_utilization_ratio', 'total_emi_per_month',
                    'amount_invested_monthly', 'monthly_balance']

to_numeric(df_credito, numeric_columns)

,age,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,...,auto loan,credit-builder loan,debt consolidation loan,home equity loan,mortgage loan,not specified,payday loan,personal loan,student loan,unknown loan
0,23,19114.12,1824.843333,3,4,3,4,3,7.0,11.27,...,1,0,0,0,0,0,0,0,0,0
1,23,19114.12,NaN,3,4,3,4,-1,NaN,11.27,...,1,0,0,0,0,0,0,0,0,0
2,-500,19114.12,NaN,3,4,3,4,3,7.0,NaN,...,1,0,0,0,0,0,0,0,0,0
3,23,19114.12,NaN,3,4,3,4,5,4.0,6.27,...,1,0,0,0,0,0,0,0,0,0
4,23,19114.12,1824.843333,3,4,3,4,6,NaN,11.27,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,25,39628.99,3359.415833,4,6,7,2,23,7.0,11.50,...,1,0,0,0,0,0,0,0,0,0
99996,25,39628.99,3359.415833,4,6,7,2,18,7.0,11.50,...,1,0,0,0,0,0,0,0,0,0
99997,25,39628.99,3359.415833,4,6,5729,2,27,6.0,11.50,...,1,0,0,0,0,0,0,0,0,0
99998,25,39628.99,3359.415833,4,6,7,2,20,NaN,11.50,...,1,0,0,0,0,0,0,0,0,0


In [11]:
#arredondar valores numéricos para 2 casas decimais para evitar overfitting
df_credito['monthly_inhand_salary'] = df_credito['monthly_inhand_salary'].round(2)
df_credito['total_emi_per_month'] = df_credito['total_emi_per_month'].round(2)
df_credito['credit_utilization_ratio'] = df_credito['credit_utilization_ratio'].round(2)
df_credito['amount_invested_monthly'] = df_credito['amount_invested_monthly'].round(2)
df_credito['monthly_balance'] = df_credito['monthly_balance'].round(2)
df_credito['annual_income'] = df_credito['annual_income'].round(2)
df_credito['changed_credit_limit'] = df_credito['changed_credit_limit'].round(2)

In [12]:
#transformar valores negativos exdrúxulos em nulos
def limit_values(dataframe, column_name, min_number):
    dataframe.loc[dataframe[column_name] <= min_number, column_name] = np.nan
    return dataframe

#coluna age -> mínimo de 17
limit_values(df_credito,'age', 17)

#coluna num_bank_accounts -> mínimo de 0
limit_values(df_credito,'num_bank_accounts', 0)

,age,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,...,auto loan,credit-builder loan,debt consolidation loan,home equity loan,mortgage loan,not specified,payday loan,personal loan,student loan,unknown loan
0,23.0,19114.12,1824.84,3.0,4,3,4,3,7.0,11.27,...,1,0,0,0,0,0,0,0,0,0
1,23.0,19114.12,NaN,3.0,4,3,4,-1,NaN,11.27,...,1,0,0,0,0,0,0,0,0,0
2,NaN,19114.12,NaN,3.0,4,3,4,3,7.0,NaN,...,1,0,0,0,0,0,0,0,0,0
3,23.0,19114.12,NaN,3.0,4,3,4,5,4.0,6.27,...,1,0,0,0,0,0,0,0,0,0
4,23.0,19114.12,1824.84,3.0,4,3,4,6,NaN,11.27,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,25.0,39628.99,3359.42,4.0,6,7,2,23,7.0,11.50,...,1,0,0,0,0,0,0,0,0,0
99996,25.0,39628.99,3359.42,4.0,6,7,2,18,7.0,11.50,...,1,0,0,0,0,0,0,0,0,0
99997,25.0,39628.99,3359.42,4.0,6,5729,2,27,6.0,11.50,...,1,0,0,0,0,0,0,0,0,0
99998,25.0,39628.99,3359.42,4.0,6,7,2,20,NaN,11.50,...,1,0,0,0,0,0,0,0,0,0


In [13]:
# tratamento da coluna credit_history_age (vamos transformar em anos)

import re

def convert_to_years(value):
    if pd.notna(value):
        match = re.match(r'(\d+) Years and (\d+) Months', value)
        if match:
            years = int(match.group(1))
            months = int(match.group(2))
            return round(years + months / 12, 1)
    return value

df_credito['credit_history_years'] = df_credito['credit_history_age'].apply(convert_to_years)
df_credito = df_credito.drop(columns=['credit_history_age'])

In [14]:
#ajustar a coluna num_of_loan
df_credito['num_of_loan'] = loan_dummies.iloc[:, 0:].sum(axis=1)

Exportação do dataset para treinamento

In [15]:
#exportação para xgboosting
df_credito.to_csv('train_xgboosting.csv', encoding='utf-8', index=False)

#exportação para svm (necessário tratar o target para ser um problema binário)
df_credito['credit_score'] = df_credito['credit_score'].map(lambda x: 0 if x == 'Poor' else 1)
df_credito.to_csv('train_svm.csv', encoding='utf-8', index=False)